In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 55.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=538f928ce7557c078bf0cd90423dba44774c976c4a78a6f7485fd309fd9775bb
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime,timedelta
import pytz

spark = SparkSession.builder\
        .appName("Test_unixTime")\
        .getOrCreate()

In [3]:
spark

In [4]:
tz_mexico = pytz.timezone('America/Mexico_City')
fecha_hoy = datetime.now(tz_mexico).strftime('%Y-%m-%d %H:%M:%S')
fecha_hoy_dt = datetime.now(tz_mexico)

In [ ]:
df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/test1.csv', header=True)

In [ ]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2|18000 |
+---------+---+----------+------+



In [ ]:
df.count()

6

In [ ]:
df.select('name','age').show()

+---------+---+
|     name|age|
+---------+---+
|    Krish| 31|
|Sudhanshu| 30|
|    Sunny| 29|
|     Paul| 24|
|   Harsha| 21|
|  Shubham| 23|
+---------+---+



In [ ]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [ ]:
df.withColumn('Age',df['age'].cast('Integer')).printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [ ]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2|18000 |
+---------+---+----------+------+



In [ ]:
print(fecha_hoy)

2022-05-22 22:07:00


In [ ]:
dff = df.withColumn('Fecha', lit(fecha_hoy))
dff.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|              Fecha|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|2022-05-22 22:07:00|
|Sudhanshu| 30|         8| 25000|2022-05-22 22:07:00|
|    Sunny| 29|         4| 20000|2022-05-22 22:07:00|
|     Paul| 24|         3| 20000|2022-05-22 22:07:00|
|   Harsha| 21|         1| 15000|2022-05-22 22:07:00|
|  Shubham| 23|         2|18000 |2022-05-22 22:07:00|
+---------+---+----------+------+-------------------+



In [ ]:
dff.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Fecha: string (nullable = false)



In [ ]:
df_unix = dff.select('Name','age','Experience','Salary',unix_timestamp(col('Fecha')).alias('fecha'))
df_unix.show()

+---------+---+----------+------+----------+
|     Name|age|Experience|Salary|     fecha|
+---------+---+----------+------+----------+
|    Krish| 31|        10| 30000|1653257220|
|Sudhanshu| 30|         8| 25000|1653257220|
|    Sunny| 29|         4| 20000|1653257220|
|     Paul| 24|         3| 20000|1653257220|
|   Harsha| 21|         1| 15000|1653257220|
|  Shubham| 23|         2|18000 |1653257220|
+---------+---+----------+------+----------+



In [ ]:
df_not_unix = df_unix.select('Name','age','Experience','Salary',from_unixtime(col('Fecha')).alias('fecha'))
df_not_unix.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|              fecha|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|2022-05-22 22:07:00|
|Sudhanshu| 30|         8| 25000|2022-05-22 22:07:00|
|    Sunny| 29|         4| 20000|2022-05-22 22:07:00|
|     Paul| 24|         3| 20000|2022-05-22 22:07:00|
|   Harsha| 21|         1| 15000|2022-05-22 22:07:00|
|  Shubham| 23|         2|18000 |2022-05-22 22:07:00|
+---------+---+----------+------+-------------------+



In [ ]:
df_convert = df_not_unix.withColumn("fecha", unix_timestamp(col('fecha')).cast('bigint'))
df_convert.show()
df_convert.printSchema()

+---------+---+----------+------+----------+
|     Name|age|Experience|Salary|     fecha|
+---------+---+----------+------+----------+
|    Krish| 31|        10| 30000|1653257220|
|Sudhanshu| 30|         8| 25000|1653257220|
|    Sunny| 29|         4| 20000|1653257220|
|     Paul| 24|         3| 20000|1653257220|
|   Harsha| 21|         1| 15000|1653257220|
|  Shubham| 23|         2|18000 |1653257220|
+---------+---+----------+------+----------+

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- fecha: long (nullable = true)



In [ ]:
df_convert_2 = df_convert.withColumn('fecha', from_unixtime(col('fecha')).cast('string'))
df_convert_2.show()
df_convert_2.printSchema()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|              fecha|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|2022-05-22 22:07:00|
|Sudhanshu| 30|         8| 25000|2022-05-22 22:07:00|
|    Sunny| 29|         4| 20000|2022-05-22 22:07:00|
|     Paul| 24|         3| 20000|2022-05-22 22:07:00|
|   Harsha| 21|         1| 15000|2022-05-22 22:07:00|
|  Shubham| 23|         2|18000 |2022-05-22 22:07:00|
+---------+---+----------+------+-------------------+

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- fecha: string (nullable = true)



In [ ]:
df.select(count('Name')).collect()

[Row(count(Name)=6)]

In [15]:
df = spark.read.csv('/ratings.csv', header=True)

In [7]:
df.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [58]:
list_users = ['1','2','3','4','5','6','7','8','9']

In [93]:
df_new = df.select(df.userId,df.movieId)
df_new.show()

+------+-------+
|userId|movieId|
+------+-------+
|     1|      1|
|     1|      3|
|     1|      6|
|     1|     47|
|     1|     50|
|     1|     70|
|     1|    101|
|     1|    110|
|     1|    151|
|     1|    157|
|     1|    163|
|     1|    216|
|     1|    223|
|     1|    231|
|     1|    235|
|     1|    260|
|     1|    296|
|     1|    316|
|     1|    333|
|     1|    349|
+------+-------+
only showing top 20 rows



In [167]:
users = df_new.\
  select('userId','movieId').\
  filter(df_new.userId.isin(list_users))

In [172]:
users2 = users.groupBy('userId').agg(count('movieId').alias('movies')).orderBy(users.userId.asc())

In [173]:
users2.show()

+------+------+
|userId|movies|
+------+------+
|     1|   232|
|     2|    29|
|     3|    39|
|     4|   216|
|     5|    44|
|     6|   314|
|     7|   152|
|     8|    47|
|     9|    46|
+------+------+



In [268]:
users_pivot = users.groupBy('userId').pivot('userId').agg(count('movieId').alias('movies')).withColumn('Usuarios',lit('Number of movies')).orderBy(users.userId.asc())
users_pivot.printSchema()

root
 |-- userId: string (nullable = true)
 |-- 1: long (nullable = true)
 |-- 2: long (nullable = true)
 |-- 3: long (nullable = true)
 |-- 4: long (nullable = true)
 |-- 5: long (nullable = true)
 |-- 6: long (nullable = true)
 |-- 7: long (nullable = true)
 |-- 8: long (nullable = true)
 |-- 9: long (nullable = true)
 |-- Usuarios: string (nullable = false)



In [273]:
otro = users_pivot.select(
    sum('1').alias('user 1'),
    sum('2').alias('user 2'),
    sum('3').alias('user 3'),
    sum('4').alias('user 4'),
    sum('5').alias('user 5'),
    sum('6').alias('user 6'),
    sum('7').alias('user 7'),
    sum('8').alias('user 8'),
    sum('9').alias('user 9'))

In [274]:
otro.show()

+------+------+------+------+------+------+------+------+------+
|user 1|user 2|user 3|user 4|user 5|user 6|user 7|user 8|user 9|
+------+------+------+------+------+------+------+------+------+
|   232|    29|    39|   216|    44|   314|   152|    47|    46|
+------+------+------+------+------+------+------+------+------+

